In [1]:
import re
import time
import pickle

import pandas as pd

from datetime import datetime

In [25]:
time_listens = []

In [26]:
with open("time_mbids.txt", 'r') as tmf:
    for line in tmf:
        user_listens = []
        line = line.strip()
        pass0 = re.sub(r'[\'u]','',line) 
        pass1 = re.sub(r'[^A-Za-z0-9,:\-\s)]','',pass0) 
        pass2 = re.sub(r'Ag', 'Aug', pass1)
        pass2 = re.sub(r'Jl', 'Jul', pass2)
        pass2 = re.sub(r'Jn', 'Jun', pass2)
        tuple_segs = pass2.split(", ")
        segs = []
        for tus in tuple_segs:
            ts = tus.split(")")
            for t in ts:  
                if t != "":
                    segs.append(t.strip())
        #print segs[:10]
        #print len(segs)
        for i in xrange(0,len(segs)-4, 4):
            listen = (segs[i], segs[i+1], segs[i+2], segs[i+3])
            user_listens.append(listen)
        time_listens.append(user_listens)
        #print user_listens[:2]
        #print user_listens[-2:]
        #break

In [27]:
print len(time_listens)

9335


In [28]:
mbid_songs = {}
with open("mbid_songs.csv", 'r') as msf:
    for line in msf:
        line = line.strip()
        segs = line.split(",")
        key = segs[0]
        track = segs[1]
        mbid_songs[key] = track

In [29]:
#listen = (segs[i], segs[i+1], segs[i+2], segs[i+3])
#listen = (date, time, unixtime, mbid)
time_data = []
count = 0
for user in time_listens[:6424]:
    user_data = []
    for listen in user:
        hour = int(listen[1].split(":")[0])
        date = datetime.strptime(listen[0], '%d %b %Y')
        unix_date = datetime.fromtimestamp(int(listen[2]))#.strftime('%Y-%m-%d %H:%M:%S')
        song = mbid_songs[listen[3]]
        if song != "":
            user_data.append((date.weekday(), hour, unix_date.weekday(), unix_date.hour, song))
    time_data.append(user_data)

In [30]:
print time_data[0][-2:], "\n", time_data[9][-2:]
print len(time_data[0])
print len(time_data[1])

[(3, 12, 3, 8, 'SODZHBJ1311AFE77FE'), (3, 12, 3, 8, 'SOOYCSZ132C0809D8F')] 
[(3, 0, 2, 19, 'SOKOGAV144B8052AF9'), (3, 0, 2, 19, 'SOFTXXS12AB0179F6A')]
1579
1496


In [31]:
with open("timedata_songs.txt", 'w') as tsf:
    for user in time_data:
        for listen in user:
            outstr = ",".join(str(l) for l in listen)
            outstr += " " #space separate listen instances
            tsf.write(outstr)
        tsf.write("\n") #one line per user as usual

In [9]:
def unpickle(filename):
    ''' open file '''
    with open(filename, 'r') as picklefile:
        old_data = pickle.load(picklefile)
    return old_data

In [10]:
basedir = "./pkls/"

In [11]:
kmeans = unpickle(basedir + "kmeans40.pkl")

In [12]:
with open(basedir + "zscore_df_csv.csv", 'r') as tf:
    tracks = pd.read_csv(tf, index_col=0)

In [13]:
with open(basedir + "titles_csv3.csv", 'r') as tf:
    titles = pd.read_csv(tf, index_col=0)

In [14]:
with open("mismatch_tracks.txt", 'r') as mtf:
    bad_tracks = mtf.read().split()
btset = set(bad_tracks)

In [32]:
#listen = (date.weekday(), hour, unix_date.weekday(), unix_date.hour, song)
#listen = (day, hour, unix_day, unix_hour, song)
timedata_clusters = []
t1 = time.time()
known_songs = set(titles.song_id.values)
for user in time_data:
    new_listens = []
    for listen in user:
        song = listen[4]
        if song in known_songs:
            trackmatches = titles[titles["song_id"] == song]
            trackmatches = trackmatches["track_id"].values #could be more than one
            for tm in trackmatches:
                if tm not in btset:
                    track = tracks[tracks["track_id"] == tm]
                    track = track.iloc[:,1:] #leave out track_id
                    cluster = kmeans.predict(track)
                    listen_cluster = (listen[0],listen[1],listen[2],listen[3],cluster[0])
                    new_listens.append(listen_cluster)
    timedata_clusters.append(new_listens)
print "predict time:", time.time()-t1

predict time: 82.2838449478


In [33]:
print len(timedata_clusters)
print len(timedata_clusters[8])

6424
11


In [58]:
limit = 20
listencount = 0
for i in range(len(timedata_clusters)):
    if len(timedata_clusters[i]) >= limit:
        listencount += 1
print listencount

220


In [36]:
with open("timedata_clusters.txt", 'w') as tsf:
    for user in timedata_clusters:
        for listen in user:
            outstr = ",".join(str(l) for l in listen)
            outstr += " " #space separate listen instances
            tsf.write(outstr)
        tsf.write("\n") #one line per user as usual

In [59]:
#listen = (day, hour, unix_day, unix_hour, cluster)
limit = 20
user_hourlistens = []
num_hours = 24
for user in timedata_clusters:
    if len(user) >= limit:#user has at least 20 listens
        hourlistens = {}#[[] for h in range(num_hours)]
        for listen in user:
            if listen[1] in hourlistens:
                hourlistens[listen[1]].append(listen[4])
            else:
                hourlistens[listen[1]] = [listen[4]]
        if hourlistens:
            user_hourlistens.append(hourlistens)
    

In [61]:
print len(user_hourlistens)

220


In [60]:
print user_hourlistens[:50]

[{1: [36], 2: [36, 36, 34], 3: [36], 4: [36], 6: [36], 9: [36, 36, 3], 11: [26], 12: [3], 13: [36], 14: [36, 36, 32], 18: [36, 37, 27], 19: [36]}, {0: [8], 3: [27], 14: [5, 2], 15: [2, 34], 16: [34], 18: [34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34], 19: [34, 34]}, {6: [5], 7: [25, 5, 5, 5, 5], 8: [5, 25, 5, 5], 10: [5], 14: [5], 15: [5], 16: [5, 5], 17: [5, 5], 18: [5, 5, 5, 5, 5], 19: [5]}, {0: [5, 2], 1: [2], 2: [26], 3: [2, 2], 9: [5, 2], 11: [26], 12: [2], 13: [3, 5], 16: [34], 17: [27, 2, 5, 2, 26], 20: [5, 5], 22: [5], 23: [5, 34, 34]}, {1: [26], 2: [15], 3: [15], 4: [27], 5: [15, 27, 15], 7: [15, 15], 8: [15, 15, 15, 15, 2], 9: [15], 10: [15, 15], 15: [15], 19: [14], 23: [15]}, {7: [26], 8: [17], 9: [26, 26, 26, 17, 2], 10: [17, 5, 26], 11: [26, 17, 26, 17, 26], 12: [26], 13: [26], 14: [26], 15: [26], 16: [26, 26], 17: [26, 26], 18: [26, 17], 19: [17, 2], 21: [26, 26]}, {1: [34, 34, 34, 34], 2: [34, 34, 34, 34, 34], 3: [34], 5: [34, 34, 34, 34], 16: [34, 34], 17: [34, 34